# Preamble 
L'objectif de ce document est de présenter un algorithme de détermination de l'angle d'une remorque à l'arrière d'un véhicule équipé d'une caméra vidéo.


# import
Le programme écrit en python s'appuie sur plusieurs bibliothèques existantes standard.

In [1]:
import cv2
print(cv2.__version__)
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi
import pandas as pd
import os
from numba import jit
from scipy.signal import find_peaks

4.1.0


# File selection

In [2]:
path = "/Users/oliviermanette/Desktop/trailer detection challenge/data/P473_Arizona_Day_Asphalt_Close_To_Sunset_dry_Nominal_8300lx"
os.chdir(path)

In [ ]:
pwd

'/Users/oliviermanette/Desktop/trailer detection challenge/data/P473_Arizona_Day_Asphalt_Close_To_Sunset_dry_Nominal_8300lx'

In [ ]:
ls

P473_Arizona_Day_Asphalt_Close_To_Sunset_dry_Nominal_8300lx.avi*
P473_Arizona_Day_Asphalt_Close_To_Sunset_dry_Nominal_8300lx.dat_GT.csv*


In [ ]:
#fileName='W420_ES_Hi_Snow_Slush_Asphalt_28klux.avi'
fileName = 'P473_Arizona_Day_Asphalt_Close_To_Sunset_dry_Nominal_8300lx.avi'

# Test Video Loop

In [ ]:
cap = cv2.VideoCapture(fileName)  # load the video
while (cap.isOpened()):  # play the video by reading frame by frame
    ret, frame = cap.read()
    if ret == True:
        # optional: do some image processing here
        cv2.imshow('frame', frame)
        # show the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            #if 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

# Variables globales
Pour des raisons de lisibilité du code, l'ensemble des variables locales seront précédés du préfixe 'l' afin de les différencier des variables globales qui n'ont pas de préfixe.
## Type de données de position

In [ ]:
posType = np.dtype([('x', 'u1'), ('y', 'u2')])

## Type de données de Neurones
### Neurone sensoriel à champs récepteur

In [ ]:
NeuronType = np.dtype([('longueur', 'u1'), ('angle', 'f4'), ('weight', 'f4'),
                       ('precision', 'f4'), ('xPos', 'u1'), ('yPos', 'u2'),
                       ('groupID', 'u1'), ('layer', 'f4')])

In [ ]:
moveType = np.dtype([('longueur', 'u1'), ('angle', 'f4'), ('weight', 'f4'),
                        ('xPos', 'u1'), ('yPos', 'u2')])

## Taille des champs récepteurs neuronaux

In [ ]:
tailleField = 7

In [ ]:
NOPIXELVALUE = 999

# Fonctions
## Calcul d'un neurone champ moyen
A partir d'une liste de neurones, il retourne le neurone moyen

In [ ]:
@jit(nopython=True, parallel=True)
def getAvgFieldNeuron(lNeuronList, typeList=NeuronType):
    lNeurons = np.zeros(1, dtype=typeList)
    lpNeurons = pd.DataFrame(lNeurons)
    lpNeurons['longueur'] = int(lNeuronList.longueur[0:1])
    lpNeurons['angle'] = float(
        np.sum((lNeuronList.angle * lNeuronList.weight) /
               np.sum(lNeuronList.weight)))
    lpNeurons['weight'] = float(
        np.sum((lNeuronList.weight * lNeuronList.weight) /
               np.sum(lNeuronList.weight)))
    lpNeurons['precision'] = float(
        np.sum((lNeuronList.precision * lNeuronList.weight) /
               np.sum(lNeuronList.weight)))
    lpNeurons['xPos'] = np.around(
        np.sum((lNeuronList.xPos * lNeuronList.weight)) /
               np.sum(lNeuronList.weight))
    lpNeurons['yPos'] = np.around(
        np.sum((lNeuronList.yPos * lNeuronList.weight)) /
               np.sum(lNeuronList.weight))
    return lpNeurons

## Matrice des directions
Afin de faciliter le calcul des angles des pixels, une matrice de poids est générée afin d'appliquer à chaque pixel centré sur un champs récepteur un poids correspondant à l'angle d'une ligne passant par ce centre.
Voici comment les angles sont représentés <br> <b>IMAGE</b>

In [ ]:
@jit(nopython=True, parallel=True)
def fillAngleMat(lSize):
    lOutput = np.zeros((lSize, lSize))
    lOffset = int(np.floor(lSize / 2))
    for lX in range(0, lSize):
        for lY in range(0, lSize):
            if (lX - lOffset) == 0:
                lOutput[lX, lY] = 90
            else:
                lOutput[lX, lY] = 0.01 + np.around(
                    np.arctan((lY - lOffset) / (lOffset - lX)) / pi * 180, 2)
    lOutput[lOffset, lOffset] = 0
    return lOutput

## Fonction d'activation des neurones
Chaque neurone retourne une valeur comprise entre 0 et 255 qui reflète son niveau d'activation. Cette activation reflète le niveau de confiance que le neurone a sur le lien existant entre sa fonction de base et les pixels reçus dans son champs récepteur. Plus les pixels sont organisés de façon à former une ligne avec l'angle correspondant à la fonction de base du neurone et plus ce dernier sera activé. Comme on ne souhaite pas obtenir une activation de valeur infinie, on utilise donc une fonction sigmoide qui s'applique à l'écart-type des angles supposés. 

In [ ]:
@jit(nopython=True, parallel=True)
def sigmoidActivationFctN1(lActivationVector):
    lDenom = (1 + np.exp(0.1 * (np.abs(np.std(lActivationVector)) - 30)))
    return 255 / lDenom

## Création d'une liste de neurones à champs récepteurs

In [ ]:
@jit(nopython=True, parallel=True)
def getNeuronActivationList(idxX,
                            idxY,
                            size,
                            frameE,
                            nbPixelPts,
                            lNeuronType=NeuronType,
                            lVerbose=False):
    # Il doit y avoir un nombre minimal de pixels
    # dans le champs récepteur du neurone
    lCriterion = nbPixelPts >= size
    
    # Création d'un tableau panda de neurones
    nbNeurons = sum(lCriterion) # taille du tableau
    lNeurons = np.zeros(nbNeurons, dtype=lNeuronType)
    lpNeurons = pd.DataFrame(lNeurons)
    lpNeurons['longueur'] = size
    

    # Variable correspondant à un demi field
    lHalfL = int(np.floor(size / 2))
    
    # Matrice de détermination des angles
    lAngleMat = fillAngleMat(size)

    #indices X,Y de positions des pixels entourés de pixels
    newX = idxX[lCriterion]
    newY = idxY[lCriterion]
    
    # Affichage d'Informations facilitant le debugage
    if lVerbose:
        print("size :" + str(len(newX)))
        print("newX")
        print(np.min(newX))
        print(np.max(newX))
        print("newY")
        print(np.min(newY))
        print(np.max(newY))
        print()
        
    pos = 0
    lnPos = 0
    for lintX in newX:
        lintY = newY[pos]
        if (lintX - lHalfL) < 0 or (lintY - lHalfL) < 0:
            print("exceed the limit of the matrix")
            pos += 1
            continue

        # récupération de pixels à l'intérieur d'un champs récepteur
        lNeuronFieldFrame = frameE[int(lintX - lHalfL):int(lintX + lHalfL + 1),
                                   int(lintY - lHalfL):int(lintY + lHalfL +
                                                           1)] / 255

        try:
            # Calcul de l'angle à l'aide de la Matrice de détermination
            tmp = np.multiply(lAngleMat, lNeuronFieldFrame)
        except:
            print("error 10 : ")
            print("lAngleMat")
            print(lAngleMat)
            print("lNeuronFieldFrame")
            print(lNeuronFieldFrame)
            print("lintX")
            print(lintX)
            print("lintY")
            print(lintY)
            print("lHalfL")
            print(lHalfL)
            continue

        lNeuronFieldValues = tmp[np.nonzero(tmp)]
        if lVerbose:
            print("lNeuronFieldFrame :")
            print(lNeuronFieldFrame)
            print("np.multiply(lAngleMat, lNeuronFieldFrame)")
            print(tmp)
            print("lNeuronFieldValues")
            print(lNeuronFieldValues)
        if (np.mean(lNeuronFieldValues)) < 0:
            lNeuronFieldValues[lNeuronFieldValues > 89] = -90
        elif np.std(lNeuronFieldValues) > 45:
            lNeuronFieldValues[lNeuronFieldValues > 89] = -90
        if (lNeuronFieldValues.size > 0):
            lpNeurons.loc[pos, ['angle']] = np.mean(lNeuronFieldValues)
            lpNeurons.loc[pos, ['weight']] = sigmoidActivationFctN1(
                lNeuronFieldValues)
            lpNeurons.loc[pos, ['precision']] = np.std(lNeuronFieldValues)
            lpNeurons.loc[pos, ['xPos']] = lintX
            lpNeurons.loc[pos, ['yPos']] = lintY
            if (np.abs(np.mean(lNeuronFieldValues)) < 90):
                lpNeurons.loc[pos, ['layer']] = np.around(lintY +
                    np.sin(np.around(np.mean(lNeuronFieldValues)) / 180 * np.pi) *
                    lintX);
            else:
                lpNeurons.loc[pos, ['layer']] = frameE.shape[0] + lintX;
            lnPos += 1
        else:
            True  #print ("error it shouldn't be zero")
        pos += 1
    if lVerbose:
        print("nb de positions couvertes : " + str(lnPos) + " sur " + str(pos))

    return lpNeurons

In [ ]:
@jit(nopython=True, parallel=True)
def getFieldNeuronPopulation(lframe,
                             lintTailleField=tailleField,
                             typeList=NeuronType,
                             lVerbose=False):

    indices = np.where(lframe != [0])

    lTmpNbPixels = nbPixelField(indices[0], indices[1], lframe,
                                lintTailleField)

    return getNeuronActivationList(indices[0], indices[1], lintTailleField,
                                   lframe, lTmpNbPixels, typeList, lVerbose)

## Nombre de pixels actifs dans chaque champs récepteur
A partir des coordonnées des centres supposés de chaque champs récepteurs et de la taille du champs récepteur, recherche sur la frame bitmap passée en paramètres, retourne un tableau contenant le nombre de pixels allumés à l'intérieur de chacun de ces champs.

In [ ]:
@jit(nopython=True, parallel=True)
def nbPixelField(lTableX, lTableY, lFrameEdge, lintTailleField=tailleField):
    lIdx = 0
    lResults = np.zeros(lTableX.size)
    lRayon = np.floor(lintTailleField / 2)
    lTailleMaxX = lFrameEdge.shape[0]
    #lTailleMaxY = lFrameEdge.shape[1]
    lHalfX = lTailleMaxX / 3

    for lPosX in lTableX:
        lPosY = lTableY[lIdx]
        if lPosX > lHalfX and lPosX >= lRayon and (lPosX +
                                                   lRayon) < lTailleMaxX:
            lResults[lIdx] = np.sum(
                lFrameEdge[int(lPosX - lRayon):int(lPosX + lRayon + 1),
                           int(lPosY - lRayon):int(lPosY + lRayon + 1)] / 255)
        lIdx += 1
    return lResults

In [ ]:
@jit(nopython=True, parallel=True)
def getNonZero(LImg):
    return np.where(LImg != [0])

## Coordonnées de la fonction de base (ligne)

In [ ]:
@jit(nopython=True, parallel=True)
def getNFCoordinate(lNeurone,lVerbose=False):
    try:
        lintDist = int(np.floor(lNeurone.longueur / 2))
    except:
        lP1 = (0, 0)
        lP2 = (0, 0)
        return (lP1, lP2)
    if np.abs(lNeurone.angle) < 45:
        lAlpha = lNeurone.angle / 180 * pi
        lintY1 = np.around(lNeurone.yPos - lintDist * np.tan(lAlpha))
        lintX1 = lNeurone.xPos + lintDist
        lintY2 = np.around(lNeurone.yPos + lintDist * np.tan(lAlpha))
        lintX2 = lNeurone.xPos - lintDist
    else:
        lAlpha = (90 - lNeurone.angle) / 180 * pi
        if lVerbose:
            print("Angle : "+str(lNeurone.angle))
            print("yPos = "+str(lNeurone.yPos)+"xPos = "+str(lNeurone.xPos))
        lintX1 = np.around(lNeurone.xPos - lintDist * np.tan(lAlpha))
        lintY1 = lNeurone.yPos + lintDist
        lintX2 = np.around(lNeurone.xPos + lintDist * np.tan(lAlpha))
        lintY2 = lNeurone.yPos - lintDist
    lP1 = (int(lintY1), int(lintX1))
    lP2 = (int(lintY2), int(lintX2))
    if lVerbose:
        print("point 1: "+str(lP1))
        print("point 2: "+str(lP2))
        print("")
    return lP1, lP2

## Calcule la distance entre deux points

In [ ]:
@jit(nopython=True, parallel=True)
def getDistance(lx1, ly1, lx2, ly2):
    return np.sqrt(
        np.power(np.abs(lx1 - lx2), 2) + np.power(np.abs(ly1 - ly2), 2))

## Retourne les neurones les plus proches d'un point

In [ ]:
@jit(nopython=True, parallel=True)
def closestFieldNeurons(lneuronList, lposX, lposY, ldistance):
    return lneuronList[(lneuronList.xPos >= lposX - ldistance)
                       & (lneuronList.xPos <= lposX + ldistance) &
                       (lneuronList.yPos >= lposY - ldistance) &
                       (lneuronList.yPos <= lposY + ldistance)]

## Crée un neurone avec les paramètres passés

In [ ]:
@jit(nopython=True, parallel=True)
def createNeuron(llong, langle, lXpos, lYpos, lweight=255, lprecis=0, lGroup=0, llayer=0, lNType=NeuronType):
    lNeurons = np.zeros(1, dtype=lNType);
    lpNeurons = pd.DataFrame(lNeurons);
    lpNeurons['longueur'] = llong;
    lpNeurons['angle'] = langle;
    lpNeurons['weight'] = lweight;
    lpNeurons['precision'] = lprecis;
    lpNeurons['xPos'] = lXpos;
    lpNeurons['yPos'] = lYpos;
    lpNeurons['groupID'] = lGroup;
    lpNeurons['layer'] = llayer;
    return lpNeurons;

## Dessine les fonctions de base des neurones sur un bitmap

In [ ]:
@jit(nopython=True, parallel=True)
def drawFieldNeurons(lNeuronList,
                     lBitmap,
                     lVerbose=False,
                     lGroupMember=0,
                     lLayer=0,
                     lLayerRange=0,
                     lAngle=0,
                     lAngleRange=0):
    lInitShow = 8
    if lVerbose:
        print ("WARNING 50 : In Verbose Mode all data are not displayed on frame!");
        lInitShow = 0
    lIndexPassOver = lInitShow
    for _, lNeuron in lNeuronList.iterrows():
        if lGroupMember > 0:
            if lNeuron.groupID != lGroupMember:
                continue
        if lLayer != 0:
            if (lNeuron.layer < lLayer - lLayerRange) or (
                    lNeuron.layer > lLayer + lLayerRange):
                continue
        if lAngle != 0:
            if (lNeuron.angle < lAngle - lAngleRange) or (
                    lNeuron.angle > lAngle + lAngleRange):
                continue
        lCoord = getNFCoordinate(lNeuron, lVerbose)
        if lVerbose:
            print("Neuron : "+str(lNeuron));
            print("Coordonnées : "+str(lCoord));
        if lIndexPassOver > 7:
            lIndexPassOver = lInitShow
            try:
                cv2.line(
                    lBitmap,
                    lCoord[0],
                    lCoord[1],
                    #        (255, 255, 255), 1)
                    (int(lNeuron.weight), int(
                        lNeuron.weight), int(lNeuron.weight)),
                    1)
            except:
                True
        if lVerbose:
            lIndexPassOver += 1
    return lBitmap

## Find neuronal groups
Un groupe neuronal est un ensemble de neurone dont les champs récepteurs sont complémentaires les uns des autres. Pour faire  partie d'un champs récepteur, deux conditions doivent être réunies.
<b>(A compléter)</b>
### Translation
Retourne les coordonnées d'un point translaté d'une certaine distance avec un certain angle. Cette fonction demande un angle, une distance et les coordonnées d'un point de départ. Il retourne ensuite les coordonnées après translation.

In [ ]:
@jit(nopython=True, parallel=True)
def moveCoordDeg(langle, lstartX, lstartY, ldistance, lVerbose=False):
    if lVerbose:
        ##DEBUG
        print("* moveCoordDeg(" + str(float(langle)) + "," +
              str(int(lstartX)) + "," + str(int(lstartY)) +
              "," + str(int(ldistance))+")")
        ##DEBUG
    ltipX = lstartX + ldistance * np.cos(langle / 180 * pi)
    ltipY = lstartY - ldistance * np.sin(langle / 180 * pi)
    if lVerbose:
        print ("* coord ==> ("+str(ltipX)+","+str(ltipY))
    return ltipX, ltipY

Effectue le même calcul que la fonction <i>moveCoordDeg </i>  mais prend comme paramètre un neurone. Il effectue la translation en prenant comme point de départ le centre du champs récepteur et effectue un déplacement de la taille de ce champs dans la direction de la fonction de base. 

In [ ]:
@jit(nopython=True, parallel=True)
def getNextPosition(lneuroneMoyen, lVerbose): 
    return moveCoordDeg(float(lneuroneMoyen.angle), int(lneuroneMoyen.xPos),
                        int(lneuroneMoyen.yPos), int(lneuroneMoyen.longueur), lVerbose)

### Calcul des  groupes à partir d'une liste de neurones à champs récepteurs

In [ ]:
@jit(nopython=True, parallel=True)
def findGroups(lneuronList, lVerbose=False):
    # Sélection d'un nouveau numéro de Groupe (GroupID)
    lintCurrentGroupID = 0
    lintNbGroups = 0
    lIndex = 0

    ##DEBUG
    lnbNeuron = 0
    ##DEBUG
    # liste des neurones sans groupe
    lNoGroupList = lneuronList[lneuronList.groupID == 0]

    while lNoGroupList.shape[0] > 0:

        #Sélection d'un neurone dans la liste (ceux sans groupID ou groupID=0)
        lMoyenNeuron = lNoGroupList.iloc[0]
        lIndex = lNoGroupList.head().index.values[0]

        while True:
            #Assignation d'un nouveau numéro de GroupID en cours
            lintNbGroups += 1
            lintCurrentGroupID += 1
            if lneuronList[lneuronList.groupID ==
                           lintCurrentGroupID].shape[0] == 0:
                break

        lneuronList.loc[lIndex, ['groupID']] = lintCurrentGroupID

        #déplacement
        lnPos = getNextPosition(lMoyenNeuron, lVerbose)

        #recherche de neurones proches
        lClosestNeurons = closestFieldNeurons(
            lneuronList, lnPos[0], lnPos[1],
            int(np.floor(lMoyenNeuron.longueur / 2)))
        if lVerbose:
            print("")
            print("")
            print("Coordonnées en cours : (" + str(lnPos[0]) + "," +
                  str(lnPos[1]) + ")")

            lnbNeuron += 1
            if lClosestNeurons.shape[0] == 0:
                print("Aucun neurone a proximité pour le neurone #" +
                      str(lnbNeuron) + " aux coordonnées : (" + str(lnPos[0]) +
                      "," + str(lnPos[1]) + str(") a la distance :") +
                      str(int(np.floor(lMoyenNeuron.longueur / 2))))

            #Oui ==> retour étape 1
            lNbFindGroup = 0

        while lClosestNeurons.shape[0] != 0:
            #recherche des groupID dans cette sous-sélection
            if lClosestNeurons[lClosestNeurons.groupID> 0].shape[0] == 0:
                #Non => Assigner à tous les neurones de la sous-sélection
                #le groupID en cours => aller directement à l'étape 7
                if lVerbose:
                    print("Aucun neurone dans le groupe : " +
                          str(lintCurrentGroupID))

                for lintIdx in lClosestNeurons.head().index.values:
                    lneuronList.loc[lintIdx, ['groupID']] = lintCurrentGroupID
            else:
                #Oui
                if lVerbose:
                    ##DEBUG
                    #lNbFindGroup += 1
                    print("Trouvé " + str(lClosestNeurons[
                        lClosestNeurons.groupID > 0].shape[0]) +
                          " neurone(s) déja dans des groupes :")
                    print("Groupe en cours : " + str(lintCurrentGroupID))

                #Récupération de la liste de tous les groupID utilisés
                #Sélection du groupID le plus petit
                #(en comparant aussi avec le groupID en cours)
                lintPreviousGroupID = lintCurrentGroupID
                lintCurrentGroupID = np.min(
                    lClosestNeurons[lClosestNeurons.groupID > 0].groupID)
                if lVerbose:
                    print("Change pour le groupe #" + str(lintCurrentGroupID))
                    print("-")
                #Assigner au neurone en cours le nouveau groupe
                lneuronList.loc[lIndex, ['groupID']] = lintCurrentGroupID
                #Assigner à tous les neurones de la sous-sélection ce nouveau groupID
                for lintIdx in lClosestNeurons.head().index.values:
                    lneuronList.loc[lintIdx, ['groupID']] = lintCurrentGroupID
                    #remplacer dans la liste globale,
                    #pour chaque groupID présent dans la liste par le nouveau groupID
                    for lintGroupID in lClosestNeurons[
                            lClosestNeurons.groupID > 0].groupID:
                        lneuronList.loc[lneuronList.groupID == lintGroupID,
                                        'groupID'] = lintCurrentGroupID
                if lintPreviousGroupID == lintCurrentGroupID:
                    #si tous les neurones
                    if lClosestNeurons[lClosestNeurons.groupID >
                                       0].shape[0] == lClosestNeurons[
                                           lClosestNeurons.groupID ==
                                           lintPreviousGroupID].shape[0]:
                        break  # sortie de la boucle while
            if lVerbose:
                #Calcul du neurone Field moyen
                print("Neurones trouvé :")
                print(lClosestNeurons)
            lMoyenNeuron = getAvgFieldNeuron(lClosestNeurons)
            if lVerbose:
                print("neurone Moyen")
                print(lMoyenNeuron)
            #déplacement
            lnPos = getNextPosition(lMoyenNeuron, lVerbose)

            #recherche de neurones proches
            lClosestNeurons = closestFieldNeurons(
                lneuronList, lnPos[0], lnPos[1],
                int(np.floor(lMoyenNeuron.longueur / 2)))

        lNoGroupList = lneuronList[lneuronList.groupID == 0]
    return lneuronList

## Get the Weighted Average of the group Angle (WAGA)

In [ ]:
@jit(nopython=True, parallel=True)
def getWAGA(lNeuronList, lGroupID):
    lnl = lNeuronList[lNeuronList.groupID==lGroupID]
    return float(np.sum((lnl.angle * lnl.weight) /
               np.sum(lnl.weight)))

## Another Better way to form a group : Line Neurons
### Get main neurons forming a group

In [ ]:
@jit(nopython=True, parallel=True)
def getMainNeurons(lNeuronList,
                   Order=0,
                   lVerbose=False,
                   binSizeLayer=256,
                   binSizeAngle=45,
                  lLargeurSeuil=10):

    lMargin = pd.Series([0, 0])
    
    ######## Histogramme des Layers dans la population de neurones ####
    lLayerHist = (lNeuronList['layer'].pipe(lambda s: pd.Series(
        np.histogram(s, bins=binSizeLayer))).pipe(lambda s: pd.Series(
            s[0], index=s[1][:-1])))

    lLayerHist = lLayerHist.append(lMargin)
    lLayerHist = lMargin.append(lLayerHist)
    
    ######## Détection des pics des Layers les plus fréquents ########
    peaks, ldict = find_peaks(lLayerHist, height=tailleField/2)

    if lVerbose:
        ser = lLayerHist.to_numpy()
        plt.plot(ser)
        plt.plot(peaks, ser[peaks], "x")
        plt.plot(np.zeros_like(ser), "--", color="gray")
        plt.ylabel('frequency')
        plt.show()
        print("number of group found : " + str(len(peaks)))

    lidx2 = 0
    lCurrentOrder = 0
    while lCurrentOrder <= Order:
        if (np.sum(ldict['peak_heights']) == 0):
            print("ERROR 40 : exceed number of group found")
            return lNeuronList, lNeuronList
        try:
            lidx2 = peaks[np.argmax(ldict['peak_heights'])]
        except:
            print("ERROR 30 : no neuron found")
            continue
        lidx1 = np.argmax(ldict['peak_heights'])
        if lVerbose:
            print("Order = "+str(Order)+" & CurrentOrder =  "+str(lCurrentOrder))
            print("np.argmax(ldict['peak_heights']) -> "+str(lidx1));
            print("list of peaks : "+str(peaks));
            print("height of peaks : "+str(ldict['peak_heights']));
        ldict['peak_heights'][lidx1] = 0
        lCurrentOrder += 1

    ######### Récupération de la valeur de layer correspondant au pic ##
    lLayerPic = lLayerHist.index.values[lidx2:lidx2 + 1][0]
    if lVerbose:
        print("Layer = " + str(lLayerPic))

    ######### Récupération de la sous-population dans une certaine Layer
    lNeuronTopLayer = lNeuronList[(lNeuronList.layer >= (lLayerPic - lLargeurSeuil))
                                  & (lNeuronList.layer <= (lLayerPic + lLargeurSeuil))]
    lAngleHist = lNeuronTopLayer['angle'].pipe(lambda s: pd.Series(
        np.histogram(s, bins=binSizeAngle))).pipe(lambda s: pd.Series(
            s[0], index=s[1][:-1]))
    
    lAngleHist = lAngleHist.append(lMargin)
    lAngleHist = lMargin.append(lAngleHist)

    ######### Detection de l'angle le plus fréquent ###################
    peaks, ldictAngle = find_peaks(lAngleHist, height=tailleField/2)
    if lVerbose:
        print("number of ANGLE found : " + str(len(peaks)))
    try:
        lidx2 = peaks[np.argmax(ldictAngle['peak_heights'])]
    except:
        if lVerbose:
            print("ERROR 20 : no neuron found")
        return lNeuronTopLayer
    lAnglePic = lAngleHist.index.values[lidx2:lidx2 + 1][0]

    if lVerbose:
        print("Angle = " + str(lAnglePic))

    lNeuronTopLayer = lNeuronList[(lNeuronList.layer >= (lLayerPic - lLargeurSeuil))
                                  & (lNeuronList.layer <= (lLayerPic + lLargeurSeuil))
                                  & (lNeuronList.angle >= (lAnglePic - lLargeurSeuil))
                                  & (lNeuronList.angle <= (lAnglePic + lLargeurSeuil))];
    LNLIndex = lNeuronList[(lNeuronList.layer >= (lLayerPic - lLargeurSeuil))
                                  & (lNeuronList.layer <= (lLayerPic + lLargeurSeuil))
                                  & (lNeuronList.angle >= (lAnglePic - lLargeurSeuil))
                                  & (lNeuronList.angle <= (lAnglePic + lLargeurSeuil))].index;
    LNL2 = lNeuronList.drop(LNLIndex);

    if lVerbose:
        print("Number of neurons in this group : " + str(len(lNeuronTopLayer)))

    return lNeuronTopLayer,LNL2

### Get Line Neuron 
In order to replace a list of smaller Field Neuron

In [ ]:
@jit(nopython=True, parallel=True)
def getLineNeuron(lNeuronList, lGroupID=0, typeList=NeuronType):
    lNeurons = getAvgFieldNeuron(lNeuronList, typeList)
    lTMPx1 = np.max(lNeuronList.xPos)
    lTMPx2 = np.min(lNeuronList.xPos)
    lTMPy1 = np.max(lNeuronList.yPos)
    lTMPy2 = np.min(lNeuronList.yPos)
    lNeurons.groupID = lGroupID;
    lNeurons.longueur = np.ceil(getDistance(lTMPx1,lTMPy1,lTMPx2,lTMPy2))
    return lNeurons

## Get a population of Line Number
### Get Group Number in Field Neurons List
Pour faire une boucle sur l'ensemble des neurones Field, il faut connaitre le nombre de groupes principaux il est possible de former.

In [ ]:
@jit(nopython=True, parallel=True)
def getGroupNumber(lNeuronList,
                   lVerbose=False,
                   binSizeLayer=256):   
    
    lMargin = pd.Series([0, 0])
    
    ######## Histogramme des Layers dans la population de neurones ####
    lLayerHist = (lNeuronList['layer'].pipe(lambda s: pd.Series(
        np.histogram(s, bins=binSizeLayer))).pipe(lambda s: pd.Series(
            s[0], index=s[1][:-1])))
    
    lLayerHist = lLayerHist.append(lMargin)
    lLayerHist = lMargin.append(lLayerHist)
    
    ######## Détection des pics des Layers les plus fréquent ############
    peaks, _ = find_peaks(lLayerHist, height=tailleField/2);
    if lVerbose:
        print("Looking for "+str(len(peaks))+ " peaks in a loop");
    return len(peaks)

### get whole population of List Neurons

In [ ]:
@jit(nopython=True, parallel=True)
def getLineNeuronList(lNeuronList,
                      MaxNbNeurons=0,
                      lVerbose=False,
                      binSizeLayer=256,
                      binSizeAngle=45,
                      lNeuronType=NeuronType):
    if MaxNbNeurons == 0:
        nbNeurons = getGroupNumber(lNeuronList, lVerbose, binSizeLayer)
    else:
        nbNeurons = MaxNbNeurons;
    lNeurons = np.zeros(nbNeurons, dtype=lNeuronType)
    lpNeurons = pd.DataFrame(lNeurons)

    LTMPNeurons = lNeuronList
    for lTmpI in range(0, nbNeurons):         
        lTMPNeuronLIST, LTMPNeurons = getMainNeurons(LTMPNeurons, 0, lVerbose,binSizeLayer, binSizeAngle);
        
        lTmpNeuron = getLineNeuron(lTMPNeuronLIST, lTmpI, lNeuronType);
        lTmpNeuron = pd.DataFrame(lTmpNeuron) 
        if lVerbose:
            print(getWAGA(lTMPNeuronLIST,0))
        lpNeurons.loc[lTmpI, ['longueur']] = lTmpNeuron.longueur[0]
        lpNeurons.loc[lTmpI, ['angle']] = lTmpNeuron.angle[0]
        lpNeurons.loc[lTmpI, ['weight']] = lTmpNeuron.weight[0]
        lpNeurons.loc[lTmpI, ['precision']] = lTmpNeuron.precision[0]
        lpNeurons.loc[lTmpI, ['xPos']] = lTmpNeuron.xPos[0]
        lpNeurons.loc[lTmpI, ['yPos']] = lTmpNeuron.yPos[0]
        lpNeurons.loc[lTmpI, ['groupID']] = lTmpNeuron.groupID[0]
        lpNeurons.loc[lTmpI, ['layer']] = len(lTMPNeuronLIST)#lTmpNeuron.layer[0]

    return lpNeurons;

## Get smaller frame from Line Neuron

In [ ]:
@jit(nopython=True, parallel=True)
def getSmallerFrame(lLineNeuron, frame, lRessert= False, lVerbose=False):
    ((lTMPx1,lTMPy1),(lTMPx2,lTMPy2)) = getNFCoordinate(lLineNeuron)
    if lVerbose:
        print (((lTMPx1,lTMPy1),(lTMPx2,lTMPy2)))
    if lRessert:
        lMargin = 0.5;
    else:
        lMargin = 2 * tailleField
    if lTMPx1<=lTMPx2:
        lTMPx1 = int(lTMPx1 - lMargin)
        lTMPx2 = int(lTMPx2 + lMargin)
    else:
        lTMPx1 = int(lTMPx2 - lMargin)
        lTMPx2 = int(lTMPx1 + lMargin)
    if lTMPy2 <= lTMPy1:
        lTMPy1 = int(lTMPy1 + lMargin)    
        lTMPy2 = int(lTMPy2 - lMargin)
    else:
        lTMPy1 = int(lTMPy2 + lMargin)    
        lTMPy2 = int(lTMPy1 - lMargin)
    if lVerbose:
        print (((lTMPx1,lTMPy1),(lTMPx2,lTMPy2)))
    return frame[lTMPy2:lTMPy1,lTMPx1:lTMPx2]

## Get the errors

Une erreur est l'écart existant entre une frame originale et sa reconstruction à partir de neurones Field ou Line. Afin de calculer l'erreur, il faut faire la différence entre la frame originale et la frame reconstruite. La somme des neurones restants constituent l'erreur brut. Mais cette erreur est encore plus pertinente si elle est relative au nombre total de pixels que la frame possédait initialement. On obtient ainsi une valeur flottante comprise entre 0 et 1 si l'erreur est inférieur à la frame originale voire supérieure à 1 si la reconstruction empire la situation.


In [ ]:
@jit(nopython=True, parallel=True)
def getNeuroneReceptiveField(lmnFrame, lNeuron, lVerbose=False):
    lBitmapNN = np.zeros((lmnFrame.shape[0],lmnFrame.shape[1]))
    
    lTMPNeuronMini = lNeuron;
    lTMPNeuronMini.xPos = int(np.floor(lmnFrame.shape[0]/2))
    lTMPNeuronMini.yPos = int(np.floor(lmnFrame.shape[1]/2))
    
    lBitmapNN = drawFieldNeurons(lTMPNeuronMini, lBitmapNN,0,0)
    lBitmapNN[lBitmapNN>0]=1
    return lBitmapNN

In [ ]:
@jit(nopython=True, parallel=True)
def getError(frameBase, frameTest, lVerbose=False):
    lOrigBase = np.sum(frameBase)
    lBrutError = np.sum(np.abs(frameBase - frameTest))

    if lOrigBase != 0:
        lRelativeError = lBrutError / lOrigBase
    else:
        lRelativeError = NOPIXELVALUE
        if lVerbose:
            print(
            "WARNING 60 : No pixel in the original frame at these coordinates"
            )

    if lVerbose:
        print(lTMPNeuronMini.xPos)
        print(lTMPNeuronMini.yPos)
        print(lOrigBase)
        print(np.sum(frameTest))
        print(lBrutError)
        print(lRelativeError)
    return lRelativeError

### Get Mini Error
L'erreur mini est celle entre la reconstruction donnée par <b>un neurone unique </b> et une mini frame originale, correspondant juste au field de ce neurone. Le field d'un neurone field est carré de dimension impaire pour avoir un pixel au centre. Tandis que le champs de perception d'un neurone ligne est rectangulaire car il correspond à la combinaison des champs de plusieurs neurones fields alignés.

In [ ]:
@jit(nopython=True, parallel=True)
def getMiniError(lmnFrame, lNeuron, lVerbose=False):

    lBitmapNN = getNeuroneReceptiveField(lmnFrame, lNeuron, lVerbose)
    lmnFrame[lmnFrame > 0] = 1

    return getError(lmnFrame, lBitmapNN, lVerbose)

## Raffinage des neurones Lignes

L'algorithme de détermination des neurones lignes est rapide et efficace. La contrepartie c'est qu'il peut faire des erreurs. Pour réduire ce problème, un algorithme de vérification est proposé. Cette vérification est plus approfondie mais elle profite de l'avantage du faible nombre de neurones lignes par rapport au nombre de neurones field pour être plus efficace.


In [ ]:
@jit(nopython=True, parallel=True)
def checkNeuronLineList(lNeuronLineList, OriginalFrame, lVerbose=False):
    lTmpNList = lNeuronLineList
    for index, lLNeuron in lTmpNList.iterrows():
        lMiniFrame = getSmallerFrame(lLNeuron, OriginalFrame,0,lVerbose)
        
        lLNeuron = pd.DataFrame(lLNeuron).T;
        lRError1 = getMiniError(lMiniFrame, lLNeuron, lVerbose);
        
        if lVerbose:
            print(index);
            print(lLNeuron);
            print(lRError1);
        if (lRError1==NOPIXELVALUE):
            lTmpNList = lTmpNList.drop(index);
            continue

        lTmpIdx = np.where(lMiniFrame != [0])
        lnbPixels = nbPixelField(lTmpIdx[0], lTmpIdx[1], lMiniFrame,
                                   tailleField)
        lTmpFieldNeuronList = getNeuronActivationList(lTmpIdx[0], lTmpIdx[1],
                                                      tailleField, lMiniFrame,
                                                      lnbPixels)
        lNewAngle = getWAGA(lTmpFieldNeuronList,0)
        
        lLNeuron.angle = lNewAngle        
        lRError2 = getMiniError(lMiniFrame, lLNeuron, lVerbose)
        
        if lVerbose:
            print(lLNeuron)
            print(lRError2)
        if (lRError1>lRError2):
            lTmpNList.loc[index, ['angle']] = lNewAngle
    return lTmpNList

## Masque pour les frames

Permet d'ignorer les neurones fields ou pixels non directement impliqués, le masque est utile car du fait de la réalisation des matrices dans l'architecture d'un ordinateur, il est complexe de créer des matrices obliques pour regarder un rectangle oblique. Ce n'est pas impossible, mais nécessite des calculs de rotation et de translation afin de déterminer les correspondances. Une alternative moins couteuse sur le plan computationnel est de conserver l'orientation d'origine. L'inconvénient c'est que se retrouvent inclus des pixels qui ne devraient pas l'être au niveau des coins. Or ces coins peuvent avoir une taille plus importante que la zone réellement utile. Surtout dans des angles aux alentours de 45°. La manière la plus simple est donc d'utiliser un masque qui ne va conserver que la zone utile et supprimer les pixels indésirables.

In [ ]:
@jit(nopython=True, parallel=True)
def getMask(lFrame, lMargin, lVerbose=False):
    lBitMapMask = np.zeros((lFrame.shape[0],lFrame.shape[1]))
    lTmpIdx = np.where(lFrame != [0])
    if lVerbose:
        print("taille de l'index : "+str(len(lTmpIdx[0])))
    for i in range(0, len(lTmpIdx[0])):
        if lVerbose:
            print(
                str(lTmpIdx[0][i] - lMargin) + ":" + str(lTmpIdx[0][i] + lMargin))
            print(
                str(lTmpIdx[1][i] - lMargin) + ":" + str(lTmpIdx[1][i] + lMargin))
        try:
            if lVerbose:
                print(lBitMapMask[lTmpIdx[0][i] - lMargin:lTmpIdx[0][i] +
                            lMargin, lTmpIdx[1][i] - lMargin:lTmpIdx[1][i] +
                            lMargin])
            lBitMapMask[lTmpIdx[0][i] - lMargin:lTmpIdx[0][i] +
                        lMargin, lTmpIdx[1][i] - lMargin:lTmpIdx[1][i] +
                        lMargin] = 1
        except:
            continue
    return lBitMapMask

## Tableau des déplacements

In [ ]:
@jit(nopython=True, parallel=True)
def getVariation(lnewFrame,
                 lLineNeuronList,
                 lMoveTable,
                 lVerbose=False,
                 lMargin=tailleField
                 ):

    for i in range(0, len(lLineNeuronList)):

        # 0 : modifier le neurone pour
        # obtenir la nouvelle position et orientation
        # à partir du tableau des déplacements lMoveTable
        # ++++> TODO

        # 1 : obtenir une mini Frame
        lMiniFrame0 = getSmallerFrame(lLineNeuronList.loc[i], lnewFrame)

        # 2 : le champs récepteur du neurone Ligne en cours
        currentLineNeuron = pd.DataFrame(lLineNeuronList.loc[i]).T
        lNeuronRF = getNeuroneReceptiveField(lMiniFrame0, currentLineNeuron)

        # 3 : Calculer le masque à partir du champs récepteur
        lCurrentMask = getMask(lNeuronRF, lMargin)

        # 4 : Appliquer le masque
        lMiniFrame0 = np.multiply(lMiniFrame0, lMiniFrame0)
        # rationaliser les données
        lMiniFrame0[lMiniFrame0 > 0] = 255

        # 5 : Obtenir une population de neurones Field Points à l'intérieur du Field Ligne
        lCurrentFNPop = getFieldNeuronPopulation(lMiniFrame0)

        # 6 : Obtenir le neurone field moyen
        # qui nous donne les orientations et positions
        lCurrentAfN = getAvgFieldNeuron(lCurrentFNPop)

        lMoveTable.loc[i, 'angle'] += lCurrentAfN.angle.to_numpy(
        ) - currentLineNeuron.angle.to_numpy()
        
        lMoveTable.loc[i, 'weight'] += lCurrentAfN.weight.to_numpy(
        ) - currentLineNeuron.weight.to_numpy()
        
        lMoveTable.loc[i, 'xPos'] += lCurrentAfN.xPos.to_numpy(
        ) - currentLineNeuron.xPos.to_numpy()
        
        lMoveTable.loc[0, 'yPos'] += lCurrentAfN.yPos.to_numpy(
        ) - currentLineNeuron.yPos.to_numpy()

    return lMoveTable

# Video Loop

In [ ]:
kernelSize = 21  # Kernel Bluring size

# Edge Detection Parameter
parameter1 = 20
parameter2 = 40
intApertureSize = 1

#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(fileName)
lCounter = 0
while (cap.isOpened()):
    # Capture frame-by-frame
    ret, Cannyframe = cap.read()
    if ret == True:
        # Our operations on the frame come here
        if lCounter == 1:
            Cannyframe = cv2.GaussianBlur(Cannyframe, (kernelSize, kernelSize), 0, 0)
            Cannyframe = cv2.Canny(Cannyframe, parameter1, parameter2,
                              intApertureSize)  # Canny edge detection
            lCounter = 0
            break
        lCounter += 1

        # Display the resulting frame
        cv2.imshow('Edges Video', Cannyframe)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # press q to quit
            break
    else:
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

# Sandbox
## Toy data Generator

In [ ]:
def generateToy(lType=1, lHauteur=80, lLargeur=128,lepaisseur=1):
    lFrame = 0
    if lType == 1:
        lFrame = np.zeros((lHauteur, lLargeur))
        lFrame[:, int((lLargeur-lepaisseur) / 2):int((lLargeur+lepaisseur) / 2)] = 255
    elif lType == 2:
        lFrame = np.zeros((lHauteur, lLargeur))
        lFrame[int((lHauteur-lepaisseur) / 2):int((lHauteur+lepaisseur)/2), :] = 255
    elif lType == 3:
        lFrame = np.zeros((lHauteur, lLargeur))
        cv2.line(lFrame, (int(lLargeur / 3), lHauteur),
                 (int(2 * lLargeur / 3), 0), (255, 255, 255), lepaisseur)
    elif lType == 4:
        lFrame = np.zeros((lHauteur, lLargeur))
        cv2.rectangle(lFrame,
                      (int(lLargeur / 128 * 10), int(lHauteur / 80 * 30)),
                      (int(lLargeur / 128 * 30), int(lHauteur / 80 * 50)),
                      (255, 255, 255), lepaisseur)
        pts = np.array([[int(lLargeur / 128 *64),
                         int(lHauteur / 80 * 30)],
                        [int(lLargeur / 128 * 76),
                         int(lHauteur / 80 * 50)],
                        [int(lLargeur / 128 * 53),
                         int(lHauteur / 80 *50)]], np.int32)
        ts = pts.reshape((-1, 1, 2))
        cv2.polylines(lFrame, [pts], True, (255, 255, 255), lepaisseur)
        cv2.circle(lFrame,
                   (int(lLargeur / 128 * 107), int(lHauteur / 80 * 40)),
                   int(lHauteur / 80 * 10), (255, 255, 255), lepaisseur)
    elif lType == 5:
        lFrame = np.zeros((lHauteur, lLargeur))
        #createNeuron(llong, langle, lXpos, lYpos,
        llN = int(lLargeur / 256 * 50)
        llXTmp = int(lHauteur / 160 * 100)
        lNeuronTest2 = createNeuron(llN, -75, llXTmp, int(lLargeur / 256 * 25))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, -60, llXTmp, int(lLargeur / 256 * 75))
        drawFieldNeurons(lNeuronTest2, lFrame) 
        lNeuronTest2 = createNeuron(llN, 0, llXTmp, int(lLargeur / 256 * 125))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 60, llXTmp, int(lLargeur / 256 * 175))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 75, llXTmp, int(lLargeur / 256 * 225))
        drawFieldNeurons(lNeuronTest2, lFrame)
    elif lType == 6:
        lFrame = np.zeros((lHauteur, lLargeur))
        #createNeuron(llong, langle, lXpos, lYpos,
        llN = int(lLargeur / 256 * 50)
        llXTmp = int(lHauteur / 160 * 100)
        lNeuronTest2 = createNeuron(llN, 0, llXTmp, int(lLargeur / 256 * 25))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 0, llXTmp, int(lLargeur / 256 * 75))
        drawFieldNeurons(lNeuronTest2, lFrame) 
        lNeuronTest2 = createNeuron(llN, 0, llXTmp, int(lLargeur / 256 * 125))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 0, llXTmp, int(lLargeur / 256 * 175))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 0, llXTmp, int(lLargeur / 256 * 225))
        drawFieldNeurons(lNeuronTest2, lFrame)
    elif lType == 7:
        lFrame = np.zeros((lHauteur, lLargeur))
        #createNeuron(llong, langle, lXpos, lYpos,
        llN = int(lLargeur / 256 * 50)
        llXTmp = int(lHauteur / 160 * 100)
        lNeuronTest2 = createNeuron(llN, 5, llXTmp, int(lLargeur / 256 * 25))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 5, llXTmp, int(lLargeur / 256 * 75))
        drawFieldNeurons(lNeuronTest2, lFrame) 
        lNeuronTest2 = createNeuron(llN, 0, llXTmp+2, int(lLargeur / 256 * 125))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 0, llXTmp+2, int(lLargeur / 256 * 175))
        drawFieldNeurons(lNeuronTest2, lFrame)
        lNeuronTest2 = createNeuron(llN, 0, llXTmp+2, int(lLargeur / 256 * 225))
        drawFieldNeurons(lNeuronTest2, lFrame)
    else:
        lFrame = np.zeros((lHauteur, lLargeur))
        print("First parameter should be between 1 to 7")
    return lFrame

## Playground
### Test 1
#### Generate data of type 1

In [ ]:
frame = generateToy(1,80,128,1)
imgplot = plt.imshow(frame)

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.describe()

In [ ]:
titi[0:4]

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

In [ ]:
np.max(testBitmap)

In [ ]:
lintI = 0
while (lintI < 10):
    cv2.imshow('FRAME', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # press q to quit
        break
    lintI += 1

#### Simplification des neurones

In [ ]:
np.sum(titi.memory_usage())

In [ ]:
titi.layer.hist(bins=32)

In [ ]:
toto = titi.angle*titi.layer
toto.hist(bins=32)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap,0,0,60,5)
imgplot = plt.imshow(testBitmap)

#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[0:5]

In [ ]:
titi.groupby('groupID').size().hist()

In [ ]:
resultGroup = titi.groupby('groupID').size()
resultGroup[resultGroup>10].hist()

In [ ]:
titi.describe()

In [ ]:
titi[0:4]

### Test 2
#### Generate data of type 2

In [ ]:
frame = generateToy(2,80,128,1)
imgplot = plt.imshow(frame)

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.describe()

In [ ]:
titi[0:4]

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[0:5]

In [ ]:
titi.groupby('groupID').size().hist()

In [ ]:
titi[0:4]

### Test 3
#### Generate data of type 3

In [ ]:
frame = generateToy(3,80,128,1)
imgplot = plt.imshow(frame)

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.describe()

In [ ]:
titi[0:4]

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[0:5]

In [ ]:
titi.groupby('groupID').size().hist()

### Test 4
#### Generate data of type 4

In [ ]:
frame = generateToy(4,160,256,1)
imgplot = plt.imshow(frame)

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.describe()

In [ ]:
titi[0:4]

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

#### Simplification

##### Layer

In [ ]:
lvhist = (titi['layer']
 .pipe(lambda s: pd.Series(np.histogram(s,  bins=256)))
 .pipe(lambda s: pd.Series(s[0], index=s[1][:-1]))
)
peaks,ldict = find_peaks(lvhist, height=7)
ser = lvhist.to_numpy()
plt.plot(ser)
plt.plot(peaks, ser[peaks], "x")
plt.plot(np.zeros_like(ser), "--", color="gray")
plt.ylabel('frequency')
plt.show()

In [ ]:
lidx2 = peaks[np.argmax(ldict['peak_heights'])]
lLayerPic = lvhist.index.values[lidx2:lidx2+1][0]

In [ ]:
lidx2 = peaks[3]
lLayerPic = lvhist.index.values[lidx2:lidx2+1][0]

In [ ]:
lidx2

In [ ]:
np.argmax(ldict['peak_heights'])

In [ ]:
ldict['peak_heights'][8]=0

In [ ]:
ldict['peak_heights']

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap,0,0,lLayerPic,5)
imgplot = plt.imshow(testBitmap)

In [ ]:
plt.plot(titi.angle,titi.layer,'.')
plt.show()

In [ ]:
plt.plot(titi.angle[np.abs(titi.layer)<300],titi.layer[np.abs(titi.layer)<300],'.')
plt.show()

##### Angle

In [ ]:
lNeuronTopLayer = titi[(titi.layer>=(lLayerPic-5)) &(titi.layer<=(lLayerPic+5))]
lNeuronTopLayer.angle.hist()

In [ ]:
lvhist = (lNeuronTopLayer['angle']
 .pipe(lambda s: pd.Series(np.histogram(s,  bins=45)))
 .pipe(lambda s: pd.Series(s[0], index=s[1][:-1]))
)
lvhist = lvhist.append(pd.Series([0, 0]))
ltt  = pd.Series([0, 0])
lvhist = ltt.append(lvhist)
peaks,ldict = find_peaks(lvhist, height=7)
ser = lvhist.to_numpy()
plt.plot(ser)
plt.plot(peaks, ser[peaks], "x")
plt.plot(np.zeros_like(ser), "--", color="gray")
plt.ylabel('frequency')
plt.show()

In [ ]:
lidx2 = peaks[np.argmax(ldict['peak_heights'])]
lAnglePic = lvhist.index.values[lidx2:lidx2+1][0]

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap,0,0,lLayerPic,5,lAnglePic,5)
imgplot = plt.imshow(testBitmap)

In [ ]:
lTMPNeuronList = getMainNeurons(titi,5,True)

testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(lTMPNeuronList, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(getLineNeuron(lTMPNeuronList), testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

In [ ]:
lTMPNeuronList = getLineNeuronList(titi)
print(lTMPNeuronList)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(lTMPNeuronList, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').size().hist()

In [ ]:
resultGroup = titi.groupby('groupID').size()
resultGroup[resultGroup>10].hist()

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[resultGroup>10]

In [ ]:
for lidx, langle in titi.groupby('groupID').count().sort_values('angle', ascending=False).iterrows():
    print (lidx)

In [ ]:
getWAGA(titi,6)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap,0,10)
imgplot = plt.imshow(testBitmap)

### Test 5
#### Generate data of type 5

In [ ]:
frame = generateToy(5,160,256,1)
imgplot = plt.imshow(frame) 

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.describe()

In [ ]:
titi.groupby('angle').agg(['mean', 'count'])

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

#### Simplification

In [ ]:
titi.layer.hist(bins=64)

In [ ]:
titi.angle.hist(bins=64)

In [ ]:
toto = titi.angle*titi.layer
toto.hist(bins=64)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap,0,0,-1300,500)
imgplot = plt.imshow(testBitmap)

In [ ]:
lTMPNeuronList = getLineNeuronList(titi,0)
print(lTMPNeuronList)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(lTMPNeuronList, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').size().hist()

In [ ]:
resultGroup = titi.groupby('groupID').size()
resultGroup[resultGroup>10].hist()

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[resultGroup>10]

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap,0,5)
imgplot = plt.imshow(testBitmap)

In [ ]:
getWAGA(titi,5)

### Test 6
#### Generate data of type 6

In [ ]:
frame = generateToy(6,160,256,1)
imgplot = plt.imshow(frame) 

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.groupby('angle').agg(['mean', 'count'])

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').size().hist()

In [ ]:
resultGroup = titi.groupby('groupID').size()
resultGroup[resultGroup>10].hist()

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[resultGroup>10]

#### Simplification

In [ ]:
lTMPNeuronList = getLineNeuronList(titi,0)
print(lTMPNeuronList)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(lTMPNeuronList, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

### Test 7
#### Generate data of type 7

In [ ]:
frame = generateToy(7,160,256,1)
imgplot = plt.imshow(frame) 

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.groupby('angle').agg(['mean', 'count'])

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').size().hist()

In [ ]:
resultGroup = titi.groupby('groupID').size()
resultGroup[resultGroup>10].hist()

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[resultGroup>10]

In [ ]:
getWAGA(titi,1)

#### Simplification

In [ ]:
lTMPNeuronList = getLineNeuronList(titi,0)
print(lTMPNeuronList)

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(lTMPNeuronList, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

##### Comment l'algorithme de raffinage de la population de neurones Lignes fonctionne :

In [ ]:
miniFrame = getSmallerFrame(lTMPNeuronList.loc[4], frame,0,True);
imgplot = plt.imshow(miniFrame)

In [ ]:
imgplot = plt.imshow(getMask(miniFrame,3))

In [ ]:
pd.DataFrame(lTMPNeuronList.loc[4]).T

In [ ]:
testBitmapNN = np.zeros((miniFrame.shape[0],miniFrame.shape[1]))
#testBitmapNN = np.zeros((miniFrame.shape[0],miniFrame.shape[1],3), np.uint8)
lTMPNeuronMini = pd.DataFrame(lTMPNeuronList.loc[4]).T
lTMPNeuronMini.xPos = int(np.floor(miniFrame.shape[0]/2))
#print(lTMPNeuronMini.xPos)
lTMPNeuronMini.yPos = int(np.floor(miniFrame.shape[1]/2))
#print(lTMPNeuronMini.yPos)
testBitmapNN = drawFieldNeurons(lTMPNeuronMini, testBitmapNN,0,0)
testBitmapNN[testBitmapNN>0]=255
imgplot = plt.imshow(testBitmapNN)
lOrigBase = np.sum(miniFrame/255)
print(lOrigBase)
print(np.sum(testBitmapNN/255))
lBrutError = np.sum(np.abs((miniFrame/255)-(testBitmapNN/255)));
print(lBrutError)
lRelativeError = lBrutError/lOrigBase;
print(lRelativeError)

In [ ]:
lTmpError = np.abs((miniFrame/255)-(testBitmapNN/255))
print(np.sum(lTmpError))
imgplot = plt.imshow(lTmpError)  

In [ ]:
getMiniError(miniFrame, pd.DataFrame(lTMPNeuronList.loc[4]).T)

##### Raffinage
Obtention d'une nouvelle variable globale <b>LineNeuronPop</b>

In [ ]:
LineNeuronPop =  checkNeuronLineList(lTMPNeuronList, frame)

##### Affichage du résultat 

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(LineNeuronPop, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

### Test 8 : Two Small video Frame

#### Get the first frame

In [ ]:
frame1 = generateToy(6,160,256,1)
imgplot = plt.imshow(frame1) 

#### Génération des neurones à champs récepteur

In [ ]:
titi = getFieldNeuronPopulation(frame1)

In [ ]:
titi.groupby('angle').agg(['mean', 'count'])

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

#### Simplification

In [ ]:
lTMPNeuronList = getLineNeuronList(titi,0)
print(lTMPNeuronList)

In [ ]:
testBitmap = np.zeros((frame1.shape[0],frame1.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(lTMPNeuronList, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

##### Raffinage
Obtention d'une nouvelle variable globale <b>LineNeuronPop</b>

In [ ]:
LineNeuronPop =  checkNeuronLineList(lTMPNeuronList, frame1)
print(LineNeuronPop)

##### Affichage du résultat 

In [ ]:
testBitmap = np.zeros((frame1.shape[0],frame1.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(LineNeuronPop, testBitmap,0,0)
imgplot = plt.imshow(testBitmap)

##### Création du tableau de stockage des mouvements

In [ ]:
lMoves = np.zeros(len(LineNeuronPop), dtype=moveType)
lpMoves = pd.DataFrame(lMoves)

In [ ]:
lpMoves

#### Get the Second Frame

In [ ]:
frame2 = generateToy(7,160,256,1)
imgplot = plt.imshow(frame2) 

##### petite frame

In [ ]:

lMiniFrame0 = getSmallerFrame(LineNeuronPop.loc[0], frame2)
plt.imshow(lMiniFrame0)

In [ ]:
#getNeuroneReceptiveField(lmnFrame, lNeuron, lVerbose=False)
currentLineNeuron = pd.DataFrame(LineNeuronPop.loc[0]).T
lNeuronRF = getNeuroneReceptiveField(lMiniFrame0,currentLineNeuron)
plt.imshow(lNeuronRF)

In [ ]:
lCurrentMask = getMask(lNeuronRF,3)
plt.imshow(lCurrentMask)

In [ ]:
lMiniFrame0 = np.multiply(lMiniFrame0,lMiniFrame0)
plt.imshow(lMiniFrame0)

In [ ]:
lMiniFrame0[lMiniFrame0>0]=255

In [ ]:
np.max(lMiniFrame0)

In [ ]:
lCurrentFNPop = getFieldNeuronPopulation(lMiniFrame0)

In [ ]:
lCurrentFNPop.groupby('angle').agg(['mean', 'count'])

In [ ]:
lCurrentAfN = getAvgFieldNeuron(lCurrentFNPop)

In [ ]:
lCurrentAfN

In [ ]:
currentLineNeuron

In [ ]:
len(LineNeuronPop)

In [ ]:
lpMoves.loc[0,'angle']=lCurrentAfN.angle.to_numpy() - currentLineNeuron.angle.to_numpy()
lpMoves.loc[0,'weight']=lCurrentAfN.weight.to_numpy() - currentLineNeuron.weight.to_numpy()
lpMoves.loc[0,'xPos']=lCurrentAfN.xPos.to_numpy() - currentLineNeuron.xPos.to_numpy()
lpMoves.loc[0,'yPos']=lCurrentAfN.yPos.to_numpy() - currentLineNeuron.yPos.to_numpy()

In [ ]:
lpMoves.loc[0,'angle']

In [ ]:
lMoves = np.zeros(len(LineNeuronPop), dtype=moveType)
lpMoves = pd.DataFrame(lMoves)
getVariation(frame2,LineNeuronPop,lpMoves)

### Test 9 : video frame
#### Get Video frame

In [ ]:
frame = Cannyframe
imgplot = plt.imshow(frame)

#### Génération des neurones à champs récepteur

In [ ]:
indices = np.where(frame != [0])
nbPixelsAll = nbPixelField(indices[0], indices[1], frame, tailleField)
titi = getNeuronActivationList(indices[0], indices[1], tailleField, frame,
                               nbPixelsAll)

In [ ]:
titi.describe()

In [ ]:
titi[0:4]

#### Affichage graphique du champs récepteur des neurones

In [ ]:
testBitmap = np.zeros((frame.shape[0],frame.shape[1],3), np.uint8)
testBitmap = drawFieldNeurons(titi, testBitmap)
imgplot = plt.imshow(testBitmap)

In [ ]:
lintI = 0
while (lintI < 10):
    cv2.imshow('testBitmap', testBitmap)
    if cv2.waitKey(1) & 0xFF == ord('q'): # press q to quit
        break
    lintI += 1

##### Affichage de la frame :

![Edge detection frame image, with the Canny Algorithm](frame01.png)

##### Affichage des champs récepteurs des neurones

![NeuronField Image : Le niveau de gris correspond au niveau d'activation du neurone](neuronField01.png)


#### Génération des groupes

In [ ]:
findGroups(titi);

In [ ]:
titi.groupby('groupID').size().hist()

In [ ]:
resultGroup = titi.groupby('groupID').size()
resultGroup[resultGroup>10].hist()

In [ ]:
titi.groupby('groupID').agg(['mean', 'count'])[resultGroup>130]

# Warning list

## Warning 10
Problème dans la fonction [getNeuronActivationList](#Création-d'une-liste-de-neurones-à-champs-récepteurs)

## Warning 20

## Warning 30

## Warning 40

## Warning 50

## Warning 60

